### 전이학습

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 이미지 데이터셋, 전처리, 전이학습 모듈
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights

from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.functional.classification import multiclass_accuracy

In [9]:
# 데이터 준비
img_dir = '../DATA/IMAGE'

# Resnet 전처리
preprocessing = transforms.Compose(transforms = [transforms.Resize(size = 256, interpolation = transforms.InterpolationMode.BILINEAR),
                                    transforms.CenterCrop(size = 224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))])

# 이미지 데이터셋 생성
imgDS = ImageFolder(root = img_dir, transform = preprocessing)
print(imgDS.classes, imgDS.targets, imgDS.imgs, sep = '\n')

['naeun', 'yeeun']
[0, 0, 1, 1]
[('../DATA/IMAGE\\naeun\\naeun1.jpg', 0), ('../DATA/IMAGE\\naeun\\naeun2.jpg', 0), ('../DATA/IMAGE\\yeeun\\yeeun1.jpg', 1), ('../DATA/IMAGE\\yeeun\\yeeun2.png', 1)]


In [13]:
# 데이터 로더 생성
imgDL = DataLoader(imgDS, batch_size = 2, shuffle = True, drop_last = True)
for (img, label) in imgDL: print(img.shape, label)

torch.Size([2, 3, 224, 224]) tensor([1, 0])
torch.Size([2, 3, 224, 224]) tensor([0, 1])


In [14]:
# 모델 설계 및 설정

# 사전학습된 모델 인스턴스 생성
res_model = resnet18(weights = ResNet18_Weights.DEFAULT)

# 전결합층 변경
# in feature : FeatureMap에서 받은 피처 수, out_featrues : 출력/분류 클래스 수
res_model.fc = nn.Linear(in_features = 512, out_features = 3)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\user/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [15]:
summary(model = res_model, input_size = (2, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [2, 3]                    --
├─Conv2d: 1-1                            [2, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [2, 64, 112, 112]         128
├─ReLU: 1-3                              [2, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [2, 64, 56, 56]           --
├─Sequential: 1-5                        [2, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [2, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [2, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [2, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [2, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [2, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [2, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [2, 64, 56, 56]           --
│

In [17]:
# Resnet18 Feature Module 파라미터 requires_grad = True => False 비활성화
for name, param in res_model.named_parameters():
    print(name, param.requires_grad, end = '     =====>     ')
    param.requires_grad = False
    print(param.requires_grad)

for name, param in res_model.fc.named_parameters():
    print(name, param.requires_grad, end = '     =====>     ')
    param.requires_grad = True
    print(param.requires_grad)

conv1.weight False     =====>     False
bn1.weight False     =====>     False
bn1.bias False     =====>     False
layer1.0.conv1.weight False     =====>     False
layer1.0.bn1.weight False     =====>     False
layer1.0.bn1.bias False     =====>     False
layer1.0.conv2.weight False     =====>     False
layer1.0.bn2.weight False     =====>     False
layer1.0.bn2.bias False     =====>     False
layer1.1.conv1.weight False     =====>     False
layer1.1.bn1.weight False     =====>     False
layer1.1.bn1.bias False     =====>     False
layer1.1.conv2.weight False     =====>     False
layer1.1.bn2.weight False     =====>     False
layer1.1.bn2.bias False     =====>     False
layer2.0.conv1.weight False     =====>     False
layer2.0.bn1.weight False     =====>     False
layer2.0.bn1.bias False     =====>     False
layer2.0.conv2.weight False     =====>     False
layer2.0.bn2.weight False     =====>     False
layer2.0.bn2.bias False     =====>     False
layer2.0.downsample.0.weight False     =

In [21]:
# 학습 준비
OPTIMIZER = optim.Adam(params = res_model.fc.parameters())
loss_fn = nn.CrossEntropyLoss()  # 손실 함수 정의
SCHEDULER = ReduceLROnPlateau(OPTIMIZER, mode = 'min', patience = 5)
EPOCHS = 10